#### Prepare

In [1]:
from libsvm.svmutil import *
from liblinear.liblinearutil import *
import itertools
import numpy as np
import random
import sys
import multiprocessing as mp

In [2]:
y_train, X_train = svm_read_problem("letter_train")
y_test, X_test = svm_read_problem("letter_test")

### 11~16: SVM

#### 11

In [127]:
# set "a" as positive, "not a" as negative, a=1
a = 1
y_train_1 = []
for i in range(len(y_train)):
    if y_train[i] == a:
        y_train_1.append(1)
    else:
        y_train_1.append(-1)
# linear kernel: -t=0
param = f'-s 0 -t 0 -c 1 -q'
model = svm_train(y_train_1, X_train , param)
p_label, p_acc, p_val = svm_predict(y_train_1, X_train, model)

Accuracy = 99.2286% (10419/10500) (classification)


In [128]:
# Obtain the coefficients and support vectors
coef = model.get_sv_coef()
SVs = model.get_SV()

# Compute the weight vector 'w'
w = np.zeros(len(X_train[0]))
for i in range(len(coef)):
    for j, x in SVs[i].items():
        w[j-1] += coef[i][0] * x

# Compute the norm of 'w'
w_norm = np.linalg.norm(w)
print(w_norm)

6.309673609961578


#### 12, 13

In [160]:
y_train_lst = []
a_lst = [2, 3, 4, 5, 6]
Ein_lst = []
models = []
nsv_lst = [] # number of support vectors for each model

for a in a_lst:
    y_train_a = []
    for i in range(len(y_train)):
        if y_train[i] == a:
            y_train_a.append(1)
        else:
            y_train_a.append(-1)

    # polynomial kernel: -t=1
    # degree=2: -d=2
    param = f'-s 0 -t 1 -d 2 -r 1 -g 1 -c 1 -q'

    model = svm_train(y_train_a, X_train, param)
    models.append(model)  # append model to models list
    p_label, p_acc, p_val = svm_predict(y_train_a, X_train, model)

    Ein = 1-p_acc[0]/100
    Ein_lst.append(Ein)

    # get the number of support vectors for the trained model
    nsv = model.get_nr_sv()
    nsv_lst.append(nsv)

for i, a in enumerate(a_lst):
    print(f"Ein for a={a}: {Ein_lst[i]}, Number of support vectors: {nsv_lst[i]}")

max_Ein_idx = np.argmax(Ein_lst)
max_a = a_lst[max_Ein_idx]
print(f"Max Ein: {Ein_lst[max_Ein_idx]}")
print(f"a with Max Ein: {max_a}")

min_nsv_idx = np.argmin(nsv_lst)
min_a = a_lst[min_nsv_idx]
print(f"Minimum number of support vectors: {nsv_lst[min_nsv_idx]}")
print(f"a with minimum number of support vectors: {min_a}")

Accuracy = 98.8667% (10381/10500) (classification)
Accuracy = 99.3238% (10429/10500) (classification)
Accuracy = 99.0381% (10399/10500) (classification)
Accuracy = 98.5143% (10344/10500) (classification)
Accuracy = 98.8762% (10382/10500) (classification)
Ein for a=2: 0.011333333333333306, Number of support vectors: 588
Ein for a=3: 0.006761904761904747, Number of support vectors: 368
Ein for a=4: 0.009619047619047638, Number of support vectors: 499
Ein for a=5: 0.014857142857142902, Number of support vectors: 642
Ein for a=6: 0.01123809523809527, Number of support vectors: 503
Max Ein: 0.014857142857142902
a with Max Ein: 5
Minimum number of support vectors: 368
a with minimum number of support vectors: 3


#### 14

In [154]:
# set "a" as positive, "not a" as negative, a=1
a = 7
y_train_7 = []
for i in range(len(y_train)):
    if y_train[i] == a:
        y_train_7.append(1)
    else:
        y_train_7.append(-1)

y7_test = []
for j in range(len(y_test)):
    if y_test[j] == a:
        y7_test.append(1)
    else:
        y7_test.append(-1)

C_lst = [0.01, 0.1, 1, 10, 100]
Eout_lst = []

for c in range(len(C_lst)):
    # Gaussian kernel: -t=2
    param = f'-s 0 -t 2 -g 1 -c {C_lst[c]} -q'
    model = svm_train(y_train_7, X_train , param)
    p_label, p_acc, p_val = svm_predict(y7_test, X_test, model)
    Eout = 1 - p_acc[0] / 100
    Eout_lst.append(Eout)

min_Eout = min(Eout_lst)
min_C_vals = [C_lst[i] for i in range(len(C_lst)) if Eout_lst[i] == min_Eout]
min_C = min(min_C_vals)

print(f"Values of C that result in the lowest Eout: {min_C_vals}")
print(f"Smallest C value among those with the same lowest Eout: {min_C}")

Accuracy = 95.48% (4774/5000) (classification)
Accuracy = 95.48% (4774/5000) (classification)
Accuracy = 98.58% (4929/5000) (classification)
Accuracy = 99.6% (4980/5000) (classification)
Accuracy = 99.46% (4973/5000) (classification)
Values of C that result in the lowest Eout: [10]
Smallest C value among those with the same lowest Eout: 10


#### 15: C=0.1

In [3]:
# set "a" as positive, "not a" as negative, a=1
a = 7
y_train_7 = []
for i in range(len(y_train)):
    if y_train[i] == a:
        y_train_7.append(1)
    else:
        y_train_7.append(-1)

y7_test = []
for j in range(len(y_test)):
    if y_test[j] == a:
        y7_test.append(1)
    else:
        y7_test.append(-1)

In [3]:
Eout_lst = []
gammas = [0.1, 1, 10, 100, 1000]
for g in range(len(gammas)):
    # Gaussian kernel: -t=2
    param = f'-s 0 -t 2 -g {gammas[g]} -c 0.1 -q'
    model = svm_train(y_train_7, X_train , param)
    p_label, p_acc, p_val = svm_predict(y7_test, X_test, model)
    Eout = 1 - p_acc[0] / 100
    Eout_lst.append(Eout)

min_Eout = min(Eout_lst)
min_gamma_vals = [gammas[i] for i in range(len(gammas)) if Eout_lst[i] == min_Eout]
min_gamma = min(min_gamma_vals)

print(f"Values of gamma that result in the lowest Eout: {min_gamma_vals}")
print(f"Smallest gamma value among those with the same lowest Eout: {min_gamma}")

Accuracy = 95.48% (4774/5000) (classification)
Accuracy = 95.48% (4774/5000) (classification)
Accuracy = 95.98% (4799/5000) (classification)
Accuracy = 95.48% (4774/5000) (classification)
Accuracy = 95.48% (4774/5000) (classification)
Values of gamma that result in the lowest Eout: [10]
Smallest gamma value among those with the same lowest Eout: 10


#### 16

In [4]:
num_iter = 500
rng = np.random.default_rng()
gammas = [0.1, 1, 10, 100, 1000]
gamma_counts = {gamma: 0 for gamma in gammas}  # initialize the count for each gamma to 0
iter_count = 0

for t in range(num_iter):
    indices = rng.choice(len(y_train_7), size=200, replace=False)
    X_val = [X_train[i] for i in indices]
    y_val = [y_train_7[i] for i in indices]
    X_train_sub = [X_train[i] for i in range(len(X_train)) if i not in indices]
    y_train_sub = [y_train_7[i] for i in range(len(y_train_7)) if i not in indices]
 
    min_Eval = float('inf')
    best_gamma = 0
    for gamma in range(len(gammas)):
        # train model on the training set
        param = f'-s 0 -t 2 -g {gammas[gamma]} -c 0.1 -q'
        model = svm_train(y_train_sub, X_train_sub, param)
        p_label, p_acc, _ = svm_predict(y_val, X_val, model, '-q')
        E_val = 1 - p_acc[0] / 100

        # update the best gamma and minimum validation error if necessary
        if E_val < min_Eval or (E_val == min_Eval and gamma < best_gamma):
            best_gamma = gamma
            min_Eval = E_val

    # increment the count for the best gamma
    gamma_counts[gammas[best_gamma]] += 1
    iter_count +=1
    print(iter_count, " | ", "Gamma counts:", gamma_counts)

# find the gamma with the maximum count
max_gamma_count = max(gamma_counts.values())
most_frequent_gammas = [gamma for gamma, count in gamma_counts.items() if count == max_gamma_count]

print("Final gamma counts:", gamma_counts)
print("Most frequent gamma value(s):", most_frequent_gammas)

Exception ignored on calling ctypes callback function: <function print_null at 0x00000196FBD6C0D0>
Traceback (most recent call last):
  File "c:\Users\mayda\AppData\Local\Programs\Python\Python39\lib\site-packages\libsvm\svm.py", line 60, in print_null
    def print_null(s):
KeyboardInterrupt: 


##### 試試能不能快一點

In [4]:
def train_model(gamma, X_train_sub, y_train_sub, X_val, y_val):
    param = f'-s 0 -t 2 -g {gamma} -c 0.1 -q'
    model = svm_train(y_train_sub, X_train_sub, param)
    p_label, p_acc, _ = svm_predict(y_val, X_val, model, '-q')
    E_val = 1 - p_acc[0] / 100
    return gamma, E_val

if __name__ == '__main__':
    num_iter = 500
    rng = np.random.default_rng()
    gammas = [0.1, 1, 10, 100, 1000]
    manager = mp.Manager()
    gamma_counts = manager.dict({gamma: 0 for gamma in gammas})  # initialize the count for each gamma to 0
    iter_count = 0

    with mp.Pool(processes=mp.cpu_count()) as pool:
        for t in range(num_iter):
            indices = rng.choice(len(y_train_7), size=200, replace=False)
            X_val = [X_train[i] for i in indices]
            y_val = [y_train_7[i] for i in indices]
            X_train_sub = [X_train[i] for i in range(len(X_train)) if i not in indices]
            y_train_sub = [y_train_7[i] for i in range(len(y_train_7)) if i not in indices]

            results = pool.starmap(train_model, [(gamma, X_train_sub, y_train_sub, X_val, y_val) for gamma in gammas])
            best_gamma, min_Eval = min(results, key=lambda x: x[1])

            # increment the count for the best gamma
            gamma_counts[best_gamma] += 1
            iter_count +=1
            print(iter_count, " | ", "Gamma counts:", dict(gamma_counts))
            sys.stdout.flush()
            
        pool.close()
        pool.join()

    # find the gamma with the maximum count
    max_gamma_count = max(gamma_counts.values())
    most_frequent_gammas = [gamma for gamma, count in gamma_counts.items() if count == max_gamma_count]

    print("Final gamma counts:", dict(gamma_counts))
    print("Most frequent gamma value(s):", most_frequent_gammas)


### 17~20